In [1]:
!pip install pandas==0.24.2

     |████████████████████████████████| 10.1MB 2.8MB/s 
ERROR: xarray 0.15.1 has requirement pandas>=0.25, but you'll have pandas 0.24.2 which is incompatible.
ERROR: plotnine 0.6.0 has requirement pandas>=0.25.0, but you'll have pandas 0.24.2 which is incompatible.
ERROR: mizani 0.6.0 has requirement pandas>=0.25.0, but you'll have pandas 0.24.2 which is incompatible.
ERROR: google-colab 1.0.0 has requirement pandas~=1.0.0; python_version >= "3.0", but you'll have pandas 0.24.2 which is incompatible.
  Found existing installation: pandas 1.0.3
    Uninstalling pandas-1.0.3:
      Successfully uninstalled pandas-1.0.3


In [2]:
!pip install scikit-learn==0.21.2

     |████████████████████████████████| 6.7MB 2.7MB/s 
  Found existing installation: scikit-learn 0.22.2.post1
    Uninstalling scikit-learn-0.22.2.post1:
      Successfully uninstalled scikit-learn-0.22.2.post1


In [0]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import os

from sklearn.pipeline import Pipeline
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble  import GradientBoostingClassifier, RandomForestClassifier
from sklearn.model_selection import StratifiedKFold, cross_val_score, train_test_split, KFold
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import GaussianNB
import random

from sklearn import metrics
from collections import Counter
import argparse
from sklearn.model_selection import cross_validate
from sklearn.metrics import roc_auc_score, average_precision_score, f1_score, precision_score, recall_score
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import SGDClassifier
from sklearn.model_selection import ParameterGrid
import lightgbm as lgb

In [2]:
if os.name=='posix':
    # GOOGLE COLAB SETUP

    # Load the Drive helper and mount
    from google.colab import drive

    # This will prompt for authorization.
    drive.mount('/content/drive')
    
    #2. Get the file
    data_path     = 'drive/My Drive/Colab Notebooks/adaptHAN/AOBDL_code/data'
    codes_path    = 'drive/My Drive/Colab Notebooks/adaptHAN/AOBDL_code/AOBDL_TML'


    #3. Read file as panda dataframe
    train         = pd.read_csv(f'{data_path}/formspring_cleaned_no_punkt.csv') 
else:
        #2. Get the file
    data_path     = '../data'
    codes_path    = '../AOBDL_TML'


    #3. Read file as panda dataframe
    train         = pd.read_csv(f'{data_path}/formspring_cleaned_no_punkt.csv') 

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [0]:
train.text.fillna("empty", inplace=True)

In [5]:
train[train.text.isna()]

,userid,text,mal


In [0]:
rs=42
from sklearn.model_selection import train_test_split      
X, X_test, y, y_test  = train_test_split(train.text, train.mal, stratify=train.mal, test_size=0.29, random_state=rs )
X = np.array(X)
y = np.array(y)
X_test = np.array(X_test)
y_test = np.array(y_test)

In [7]:
print(f'X {X.shape}; X_test {X_test.shape}')

X (9342,); X_test (3817,)


In [0]:
# Ridge

In [0]:
kf  = StratifiedKFold(n_splits=5, random_state=rs)
auc = []
roc = []
fscore_ = []
c   = 0

word_vectorizer   = TfidfVectorizer(
    sublinear_tf  = True,
    strip_accents = 'unicode',
    analyzer      = 'word',
    token_pattern = r'\w{1,}',
    #stop_words    = 'english',
    ngram_range   = (1, 1),
    max_features  = 7000)

In [0]:
#auc_pr = 0.093434375824395
#auc_roc = 0.02343524395
c = 0
C_parameter = np.arange(0.1, 1.1, 0.1) 

# use best C

C_parameter = [1]

In [67]:
for c_p in C_parameter:  
      for c, (train_index, val_index) in enumerate(kf.split(X, y)):

            X_train, X_val      = X[train_index], X[val_index]
            y_train, y_val      = y[train_index], y[val_index] 
            word_vectorizer.fit(X_train)
            train_word_features = word_vectorizer.transform(X_train)
            val_word_features   = word_vectorizer.transform(X_val)
            y_train             = y_train.astype('int')
            y_val               = y_val.astype('int')
            classifier          = LogisticRegression(C=c_p, solver='sag', random_state=rs)
            classifier.fit(train_word_features, y_train)
            probs               = classifier.predict_proba(val_word_features)[:,1]
            auc_roc             = roc_auc_score(y_val, probs)
            auc_pr              = average_precision_score(y_val, probs)
            
            threshold = 0.14
            #for threshold in np.arange(0.1, 0.2, 0.01):
            probs_class = probs.copy()
            probs_class[probs_class >= threshold] = 1 
            probs_class[probs_class < threshold] = 0
            precision = precision_score(y_val, probs_class) 
            recall    = recall_score(y_val, probs_class)
            fscore    = f1_score(y_val, probs_class)
            print(f' {threshold} fold {c} precision {round(precision, 3)} recall {round(recall, 3)} fscore {round(fscore,3)}')

            if len(C_parameter)==1:
                # print performance
                print(f'---------------------------------------------')
                print(f'FOLD {c}: AUC PR-C = {round(auc_pr, 3)}, AUC ROC = {round(auc_roc, 3)}')
                print(f'---------------------------------------------')
                print(f'')

            auc.append(auc_pr)
            roc.append(auc_roc)
            fscore_.append(fscore)

      if len(C_parameter)!=1:
             print(f'PARAMETER C = {c_p}')

      # print performance
      print(f'-----------------------------------------------')
      print(f'CV average: AUC PR-C = {round(np.array(auc).mean(), 3)}, AUC ROC = {round(np.array(roc).mean(), 3)}, FSCORE  = {round(np.array(fscore_).mean(), 3)}')
      print(f'-----------------------------------------------')
      print(f'')

 0.14 fold 0 precision 0.534 recall 0.521 fscore 0.527
---------------------------------------------
FOLD 0: AUC PR-C = 0.567, AUC ROC = 0.874
---------------------------------------------

 0.14 fold 1 precision 0.559 recall 0.587 fscore 0.573
---------------------------------------------
FOLD 1: AUC PR-C = 0.589, AUC ROC = 0.909
---------------------------------------------

 0.14 fold 2 precision 0.561 recall 0.65 fscore 0.602
---------------------------------------------
FOLD 2: AUC PR-C = 0.571, AUC ROC = 0.904
---------------------------------------------

 0.14 fold 3 precision 0.54 recall 0.558 fscore 0.549
---------------------------------------------
FOLD 3: AUC PR-C = 0.603, AUC ROC = 0.889
---------------------------------------------

 0.14 fold 4 precision 0.462 recall 0.55 fscore 0.502
---------------------------------------------
FOLD 4: AUC PR-C = 0.554, AUC ROC = 0.898
---------------------------------------------

-----------------------------------------------
CV av

In [68]:
# TRAIN ON WHOLE DAATA AND PREDICT ON TEST
word_vectorizer.fit(X)
train_word_features  = word_vectorizer.transform(X)
test_word_features   = word_vectorizer.transform(X_test)
classifier           = LogisticRegression(C=1, solver='sag', random_state=rs)
classifier.fit(train_word_features, y)
probs                = classifier.predict_proba(test_word_features)[:,1]
auc_roc              = roc_auc_score(y_test, probs)
auc_pr               = average_precision_score(y_test, probs)

# print performance
print(f'-----------------------------------------')
print(f'TEST: AUC PR-C = {round(auc_pr, 4)}, AUC ROC = {round(auc_roc, 4)}')
print(f'-----------------------------------------')
print(f'')

threshold = 0.14
probs_class = probs.copy()
probs_class[probs_class >= threshold] = 1 
probs_class[probs_class < threshold] = 0
precision = precision_score(y_test, probs_class) 
recall    = recall_score(y_test, probs_class)
fscore    = f1_score(y_test, probs_class)
print(f' {threshold} precision {round(precision, 3)} recall {round(recall, 3)} fscore {round(fscore,3)}')


-----------------------------------------
TEST: AUC PR-C = 0.6072, AUC ROC = 0.9058
-----------------------------------------

 0.14 precision 0.561 recall 0.602 fscore 0.58


In [0]:
# RF

In [0]:
kf  = StratifiedKFold(n_splits=5, random_state=rs)
auc = []
roc = []
fscore_ = []
c   = 0

word_vectorizer   = TfidfVectorizer(
    sublinear_tf  = True,
    strip_accents = 'unicode',
    analyzer      = 'word',
    token_pattern = r'\w{1,}',
    stop_words    = 'english',
    ngram_range   = (1, 1),
    max_features  = 7000)

In [63]:
for c, (train_index, val_index) in enumerate(kf.split(X, y)):

            X_train, X_val      = X[train_index], X[val_index]
            y_train, y_val      = y[train_index], y[val_index] 
            word_vectorizer.fit(X_train)
            train_word_features = word_vectorizer.transform(X_train)
            val_word_features   = word_vectorizer.transform(X_val)
            y_train             = y_train.astype('int')
            y_val               = y_val.astype('int')
            classifier          = RandomForestClassifier(n_estimators=600, max_depth=None, max_features='auto', 
                                    min_samples_split=2, verbose = True, n_jobs=20, random_state=rs)
            classifier.fit(train_word_features, y_train)
            probs               = classifier.predict_proba(val_word_features)[:,1]
            auc_roc             = roc_auc_score(y_val, probs)
            auc_pr              = average_precision_score(y_val, probs)

            
            # print performance
            print(f'---------------------------------------------')
            print(f'FOLD {c}: AUC PR-C = {round(auc_pr, 3)}, AUC ROC = {round(auc_roc, 3)}')
            print(f'---------------------------------------------')
            print(f'')

            auc.append(auc_pr)
            roc.append(auc_roc)
            
            threshold = 0.2
            probs_class = probs.copy()
            probs_class[probs_class >= threshold] = 1 
            probs_class[probs_class < threshold] = 0
            precision = precision_score(y_val, probs_class) 
            recall    = recall_score(y_val, probs_class)
            fscore    = f1_score(y_val, probs_class)
            print(f' {threshold} fold {c} precision {round(precision, 3)} recall {round(recall, 3)} fscore {round(fscore,3)}')

            fscore_.append(fscore)

# print performance
print(f'-----------------------------------------------')
print(f'CV average: AUC PR-C = {round(np.array(auc).mean(), 3)}, AUC ROC = {round(np.array(roc).mean(), 3)}, FSCORE  = {round(np.array(fscore_).mean(), 3)}')
print(f'-----------------------------------------------')
print(f'')

[Parallel(n_jobs=20)]: Using backend ThreadingBackend with 20 concurrent workers.
[Parallel(n_jobs=20)]: Done  10 tasks      | elapsed:    0.6s
[Parallel(n_jobs=20)]: Done 410 tasks      | elapsed:   10.7s
[Parallel(n_jobs=20)]: Done 600 out of 600 | elapsed:   15.1s finished
[Parallel(n_jobs=20)]: Using backend ThreadingBackend with 20 concurrent workers.
[Parallel(n_jobs=20)]: Done  10 tasks      | elapsed:    0.0s
[Parallel(n_jobs=20)]: Done 160 tasks      | elapsed:    0.2s
[Parallel(n_jobs=20)]: Done 410 tasks      | elapsed:    0.5s
[Parallel(n_jobs=20)]: Done 600 out of 600 | elapsed:    0.6s finished


---------------------------------------------
FOLD 0: AUC PR-C = 0.5, AUC ROC = 0.865
---------------------------------------------

 0.2 fold 0 precision 0.461 recall 0.537 fscore 0.496


[Parallel(n_jobs=20)]: Using backend ThreadingBackend with 20 concurrent workers.
[Parallel(n_jobs=20)]: Done  10 tasks      | elapsed:    0.6s
[Parallel(n_jobs=20)]: Done 160 tasks      | elapsed:    4.1s
[Parallel(n_jobs=20)]: Done 410 tasks      | elapsed:   10.2s
[Parallel(n_jobs=20)]: Done 600 out of 600 | elapsed:   14.5s finished
[Parallel(n_jobs=20)]: Using backend ThreadingBackend with 20 concurrent workers.
[Parallel(n_jobs=20)]: Done  10 tasks      | elapsed:    0.1s
[Parallel(n_jobs=20)]: Done 160 tasks      | elapsed:    0.3s
[Parallel(n_jobs=20)]: Done 410 tasks      | elapsed:    0.5s
[Parallel(n_jobs=20)]: Done 600 out of 600 | elapsed:    0.7s finished


---------------------------------------------
FOLD 1: AUC PR-C = 0.472, AUC ROC = 0.868
---------------------------------------------

 0.2 fold 1 precision 0.4 recall 0.529 fscore 0.456


[Parallel(n_jobs=20)]: Using backend ThreadingBackend with 20 concurrent workers.
[Parallel(n_jobs=20)]: Done  10 tasks      | elapsed:    0.5s
[Parallel(n_jobs=20)]: Done 160 tasks      | elapsed:    4.2s
[Parallel(n_jobs=20)]: Done 410 tasks      | elapsed:   10.3s
[Parallel(n_jobs=20)]: Done 600 out of 600 | elapsed:   14.6s finished
[Parallel(n_jobs=20)]: Using backend ThreadingBackend with 20 concurrent workers.
[Parallel(n_jobs=20)]: Done  10 tasks      | elapsed:    0.0s
[Parallel(n_jobs=20)]: Done 160 tasks      | elapsed:    0.2s
[Parallel(n_jobs=20)]: Done 410 tasks      | elapsed:    0.5s
[Parallel(n_jobs=20)]: Done 600 out of 600 | elapsed:    0.6s finished


---------------------------------------------
FOLD 2: AUC PR-C = 0.446, AUC ROC = 0.862
---------------------------------------------

 0.2 fold 2 precision 0.393 recall 0.533 fscore 0.452


[Parallel(n_jobs=20)]: Using backend ThreadingBackend with 20 concurrent workers.
[Parallel(n_jobs=20)]: Done  10 tasks      | elapsed:    0.6s
[Parallel(n_jobs=20)]: Done 160 tasks      | elapsed:    4.3s
[Parallel(n_jobs=20)]: Done 410 tasks      | elapsed:   10.6s
[Parallel(n_jobs=20)]: Done 600 out of 600 | elapsed:   15.1s finished
[Parallel(n_jobs=20)]: Using backend ThreadingBackend with 20 concurrent workers.
[Parallel(n_jobs=20)]: Done  10 tasks      | elapsed:    0.0s
[Parallel(n_jobs=20)]: Done 160 tasks      | elapsed:    0.2s
[Parallel(n_jobs=20)]: Done 410 tasks      | elapsed:    0.5s
[Parallel(n_jobs=20)]: Done 600 out of 600 | elapsed:    0.6s finished


---------------------------------------------
FOLD 3: AUC PR-C = 0.516, AUC ROC = 0.877
---------------------------------------------

 0.2 fold 3 precision 0.424 recall 0.583 fscore 0.491


[Parallel(n_jobs=20)]: Using backend ThreadingBackend with 20 concurrent workers.
[Parallel(n_jobs=20)]: Done  10 tasks      | elapsed:    0.5s
[Parallel(n_jobs=20)]: Done 160 tasks      | elapsed:    4.2s
[Parallel(n_jobs=20)]: Done 410 tasks      | elapsed:   10.6s
[Parallel(n_jobs=20)]: Done 600 out of 600 | elapsed:   15.0s finished
[Parallel(n_jobs=20)]: Using backend ThreadingBackend with 20 concurrent workers.
[Parallel(n_jobs=20)]: Done  10 tasks      | elapsed:    0.0s
[Parallel(n_jobs=20)]: Done 160 tasks      | elapsed:    0.2s
[Parallel(n_jobs=20)]: Done 410 tasks      | elapsed:    0.4s
[Parallel(n_jobs=20)]: Done 600 out of 600 | elapsed:    0.6s finished


---------------------------------------------
FOLD 4: AUC PR-C = 0.482, AUC ROC = 0.874
---------------------------------------------

 0.2 fold 4 precision 0.406 recall 0.608 fscore 0.487
-----------------------------------------------
CV average: AUC PR-C = 0.483, AUC ROC = 0.869, FSCORE  = 0.476
-----------------------------------------------



In [64]:
# TRAIN ON WHOLE DAATA AND PREDICT ON TEST
word_vectorizer.fit(X)
train_word_features  = word_vectorizer.transform(X)
test_word_features   = word_vectorizer.transform(X_test)
classifier           = RandomForestClassifier(n_estimators=600, max_depth=None, max_features='auto', 
                                    min_samples_split=2, verbose = True, n_jobs=20, random_state=rs)
classifier.fit(train_word_features, y)
probs                = classifier.predict_proba(test_word_features)[:,1]
auc_roc              = roc_auc_score(y_test, probs)
auc_pr               = average_precision_score(y_test, probs)

# print performance
print(f'-----------------------------------------')
print(f'TEST: AUC PR-C = {round(auc_pr, 3)}, AUC ROC = {round(auc_roc, 3)}')
print(f'-----------------------------------------')
print(f'')

threshold = 0.2
probs_class = probs.copy()
probs_class[probs_class >= threshold] = 1 
probs_class[probs_class < threshold] = 0
precision = precision_score(y_test, probs_class) 
recall    = recall_score(y_test, probs_class)
fscore    = f1_score(y_test, probs_class)
print(f' {threshold} precision {round(precision, 3)} recall {round(recall, 3)} fscore {round(fscore,3)}')


[Parallel(n_jobs=20)]: Using backend ThreadingBackend with 20 concurrent workers.
[Parallel(n_jobs=20)]: Done  10 tasks      | elapsed:    0.7s
[Parallel(n_jobs=20)]: Done 160 tasks      | elapsed:    5.9s
[Parallel(n_jobs=20)]: Done 410 tasks      | elapsed:   14.4s
[Parallel(n_jobs=20)]: Done 600 out of 600 | elapsed:   20.5s finished
[Parallel(n_jobs=20)]: Using backend ThreadingBackend with 20 concurrent workers.
[Parallel(n_jobs=20)]: Done  10 tasks      | elapsed:    0.1s
[Parallel(n_jobs=20)]: Done 160 tasks      | elapsed:    0.4s
[Parallel(n_jobs=20)]: Done 410 tasks      | elapsed:    0.9s


-----------------------------------------
TEST: AUC PR-C = 0.498, AUC ROC = 0.88
-----------------------------------------

 0.2 precision 0.429 recall 0.589 fscore 0.497


[Parallel(n_jobs=20)]: Done 600 out of 600 | elapsed:    1.3s finished


In [0]:
# SVM

In [74]:
penalty = ['l2', 'l1']
alpha = [0.000001, 0.00001, 0.0001, 0.001, 0.01, 0.1]
max_iter = [1000, 10000, 15000]

# best parameters
penalty = ['l2']
alpha = [0.0001]
max_iter = [1000] 

for p in penalty:
  for a in alpha:
    for i in max_iter:
      auc = []
      roc = []
      fscore_ = []
      c = 0
      # print performance
      print(f'-------------')
      print(f'penalty {p}')
      print(f'alpha {a}')
      print(f'max_iter {i}')
      print(f'-------------')
      
      for c, (train_index, val_index) in enumerate(kf.split(X, y)):

            X_train, X_val      = X[train_index], X[val_index]
            y_train, y_val      = y[train_index], y[val_index] 
            word_vectorizer.fit(X_train)
            train_word_features = word_vectorizer.transform(X_train)
            val_word_features   = word_vectorizer.transform(X_val)
            y_train             = y_train.astype('int')
            y_val               = y_val.astype('int')
            classifier          = SGDClassifier(n_jobs=20, random_state=rs, loss='log', shuffle=False, 
                                    penalty=p, alpha=a, max_iter=i)
            classifier.fit(train_word_features, y_train)
            probs               = classifier.predict_proba(val_word_features)[:,1]
            auc_roc             = roc_auc_score(y_val, probs)
            auc_pr              = average_precision_score(y_val, probs)
            
            
            #print(f'---------------------------------------------')
            #print(f'FOLD {c}: AUC PR-C = {round(auc_pr, 3)}, AUC ROC = {round(auc_roc, 3)}')
            #print(f'---------------------------------------------')
            #print(f'')

            auc.append(auc_pr)
            roc.append(auc_roc)
            threshold = 0.2
            probs_class = probs.copy()
            probs_class[probs_class >= threshold] = 1 
            probs_class[probs_class < threshold] = 0
            precision = precision_score(y_val, probs_class) 
            recall    = recall_score(y_val, probs_class)
            fscore    = f1_score(y_val, probs_class)
            #print(f' {threshold} fold {c} precision {round(precision, 3)} recall {round(recall, 3)} fscore {round(fscore,3)}')

            fscore_.append(fscore)

      # print performance
      print(f'-----------------------------------------------')
      print(f'CV average: AUC PR-C = {round(np.array(auc).mean(), 3)}, AUC ROC = {round(np.array(roc).mean(), 3)}, FSCORE  = {round(np.array(fscore_).mean(), 3)}')
      print(f'-----------------------------------------------')
      print(f'')

-------------
penalty l2
alpha 0.0001
max_iter 1000
-------------
-----------------------------------------------
CV average: AUC PR-C = 0.58, AUC ROC = 0.895, FSCORE  = 0.547
-----------------------------------------------



In [75]:
# TRAIN ON WHOLE DAATA AND PREDICT ON TEST
word_vectorizer.fit(X)
train_word_features  = word_vectorizer.transform(X)
test_word_features   = word_vectorizer.transform(X_test)
classifier           = SGDClassifier(n_jobs=20, random_state=rs, loss='log', shuffle=False, 
                                    penalty=penalty[0], alpha=alpha[0], max_iter=max_iter[0])
classifier.fit(train_word_features, y)
probs                = classifier.predict_proba(test_word_features)[:,1]
auc_roc              = roc_auc_score(y_test, probs)
auc_pr               = average_precision_score(y_test, probs)

# print performance
print(f'-----------------------------------------')
print(f'TEST: AUC PR-C = {round(auc_pr, 3)}, AUC ROC = {round(auc_roc, 3)}')
print(f'-----------------------------------------')
print(f'')

threshold = 0.2
probs_class = probs.copy()
probs_class[probs_class >= threshold] = 1 
probs_class[probs_class < threshold] = 0
precision = precision_score(y_test, probs_class) 
recall    = recall_score(y_test, probs_class)
fscore    = f1_score(y_test, probs_class)
print(f' {threshold} precision {round(precision, 3)} recall {round(recall, 3)} fscore {round(fscore,3)}')


-----------------------------------------
TEST: AUC PR-C = 0.61, AUC ROC = 0.905
-----------------------------------------

 0.2 precision 0.688 recall 0.476 fscore 0.562


In [0]:
# lightgbm

In [0]:
# muner of rounds
max_rounds = 600
stopping   = 600
verbose    = 200

# LGB parameters
lgb_params = {
    'boosting_type':     'gbdt',
    'objective':         'binary',
    'metrics':           'binary_logloss',
    'bagging_fraction':  0.9,
    'feature_fraction':  0.8,
    'lambda_l1':         0.1,
    'lambda_l2':         0.1,
    'min_split_gain':    0.01,
    'min_child_weight':  2,
    'min_child_samples': 20,
    'silent':            True,
    'verbosity':         100,
    'learning_rate':     0.1,
    'max_depth':         7,
    'num_leaves':        70,
    'scale_pos_weight':  1,
    'n_estimators':      max_rounds,
    'nthread' :          20,
    'random_state':      rs,
}

In [81]:
auc = []
roc = []
fscore_ = []
for c, (train_index, val_index) in enumerate(kf.split(X, y)):

            X_train, X_val      = X[train_index], X[val_index]
            y_train, y_val      = y[train_index], y[val_index] 
            word_vectorizer.fit(X_train)
            train_word_features = word_vectorizer.transform(X_train)
            val_word_features   = word_vectorizer.transform(X_val)
            y_train             = y_train.astype('int')
            y_val               = y_val.astype('int')
            classifier          = lgb.LGBMClassifier(**lgb_params) 
            classifier.fit(train_word_features, y_train)
            probs               = classifier.predict_proba(val_word_features)[:,1]
            auc_roc             = roc_auc_score(y_val, probs)
            auc_pr              = average_precision_score(y_val, probs)

            
            # print performance
            print(f'---------------------------------------------')
            print(f'FOLD {c}: AUC PR-C = {round(auc_pr, 3)}, AUC ROC = {round(auc_roc, 3)}')
            print(f'---------------------------------------------')
            print(f'')

            auc.append(auc_pr)
            roc.append(auc_roc)
            threshold = 0.2
            probs_class = probs.copy()
            probs_class[probs_class >= threshold] = 1 
            probs_class[probs_class < threshold] = 0
            precision = precision_score(y_val, probs_class) 
            recall    = recall_score(y_val, probs_class)
            fscore    = f1_score(y_val, probs_class)
            print(f' {threshold} fold {c} precision {round(precision, 3)} recall {round(recall, 3)} fscore {round(fscore,3)}')

            fscore_.append(fscore)

# print performance
print(f'-----------------------------------------------')
print(f'CV average: AUC PR-C = {round(np.array(auc).mean(), 3)}, AUC ROC = {round(np.array(roc).mean(), 3)}, FSCORE  = {round(np.array(fscore_).mean(), 3)}')
print(f'-----------------------------------------------')
print(f'')

---------------------------------------------
FOLD 0: AUC PR-C = 0.44, AUC ROC = 0.814
---------------------------------------------

 0.2 fold 0 precision 0.515 recall 0.421 fscore 0.464
---------------------------------------------
FOLD 1: AUC PR-C = 0.461, AUC ROC = 0.847
---------------------------------------------

 0.2 fold 1 precision 0.47 recall 0.455 fscore 0.462
---------------------------------------------
FOLD 2: AUC PR-C = 0.412, AUC ROC = 0.831
---------------------------------------------

 0.2 fold 2 precision 0.444 recall 0.458 fscore 0.451
---------------------------------------------
FOLD 3: AUC PR-C = 0.46, AUC ROC = 0.826
---------------------------------------------

 0.2 fold 3 precision 0.457 recall 0.442 fscore 0.449
---------------------------------------------
FOLD 4: AUC PR-C = 0.445, AUC ROC = 0.845
---------------------------------------------

 0.2 fold 4 precision 0.394 recall 0.433 fscore 0.413
-----------------------------------------------
CV average

In [82]:
# TRAIN ON WHOLE DAATA AND PREDICT ON TEST
word_vectorizer.fit(X)
train_word_features  = word_vectorizer.transform(X)
test_word_features   = word_vectorizer.transform(X_test)
classifier           = lgb.LGBMClassifier(**lgb_params) 
classifier.fit(train_word_features, y)
probs                = classifier.predict_proba(test_word_features)[:,1]
auc_roc              = roc_auc_score(y_test, probs)
auc_pr               = average_precision_score(y_test, probs)

# print performance
print(f'-----------------------------------------')
print(f'TEST: AUC PR-C = {round(auc_pr, 3)}, AUC ROC = {round(auc_roc, 3)}')
print(f'-----------------------------------------')
print(f'')

threshold = 0.2
probs_class = probs.copy()
probs_class[probs_class >= threshold] = 1 
probs_class[probs_class < threshold] = 0
precision = precision_score(y_test, probs_class) 
recall    = recall_score(y_test, probs_class)
fscore    = f1_score(y_test, probs_class)
print(f' {threshold} precision {round(precision, 3)} recall {round(recall, 3)} fscore {round(fscore,3)}')


-----------------------------------------
TEST: AUC PR-C = 0.497, AUC ROC = 0.844
-----------------------------------------

 0.2 precision 0.486 recall 0.484 fscore 0.485
